In [46]:
import scraper
import pandas as pd
from datetime import date, timedelta
import requests
import json
import csv
import pymongo
from dotenv import dotenv_values
import os




In [47]:
today = date.today()

In [48]:

today = date.today()

# Web scrapers
dice_df = scraper.scrape_dice()

dice_df.to_json(f'dice_{today}.json')

indeed_df = scraper.scrape_indeed()

indeed_df.to_json(f'indeed_{today}.json')



In [49]:
def get_companies(filename: str):
    with open('companies.csv', 'r') as data:
        companies = {row[0]: row[1] for row in csv.reader(data)}
    companies_clean = companies
    companies_bad = {}
    tokens = set(companies.keys())
    for token in tokens:
        if not requests.get(f'https://boards-api.greenhouse.io/v1/boards/{token}/jobs'):
            companies_clean.pop(token)
            companies_bad[token] = companies.get(token)

    if companies_bad:
        with open(f"./logs/companies_bad_{date.today()}.json", "w") as out:
            json.dump(companies_bad, out)
            
    return companies_clean

 # # Main Code
    # Finds jobs and outputs results as csv or excel file

    # Return a list of eligible jobs from a set of companies



In [50]:
# Greenhouse API
companies_file = 'companies.csv'

roles = {'developer','engineer', 'frontend', 'software', 'apprentice', 'apprenticeship', 'front-end', 'backend', 'back-end', 'jr.', 'jr' }

# Includes 'software' for titles that are just 'software engineer', etc.
levels = {'junior', 'entry-level', 'grad', 'graduate', 'apprentice', 'apprenticeship', 'software', 'entry', 'intern', 'i', '1', 'associate'}

# Optional, but helps exclude higher level positions
exclude = {'senior', 'principal' , 'sr.', 'ii', 'iii' }

criteria ={'roles': roles, 'levels': levels, 'exclude' : exclude}

companies = get_companies(companies_file)


greenhouse_df = scraper.scrape_greenhouse(companies, criteria)

In [51]:
greenhouse_df

,title,company,link,description,date,remote,greenhouse_id,greenhouse_api_url,location
0,"Software Engineer, Infrastructure",Loyal Health,https://boards.greenhouse.io/loyal/jobs/477271...,"<div class=""content-intro""><p>Loyal is an orga...",2023-03-08T18:14:34-05:00,False,4772718004,https://boards-api.greenhouse.io/v1/boards/loy...,"[{'id': 4009356004, 'name': 'Anywhere in the U..."
1,"Software Engineer, Backend",Janus Health,https://boards.greenhouse.io/janushealth/jobs/...,"<div class=""content-intro""><p>At Janus, we bel...",2023-03-11T01:37:04-05:00,True,4012385007,https://boards-api.greenhouse.io/v1/boards/jan...,"[{'id': 4000619007, 'name': 'Remote', 'locatio..."
2,Software Engineer,Stariwell,https://boards.greenhouse.io/stairwell/jobs/43...,"<div class=""content-intro""><h3>We’re expanding...",2022-06-02T14:21:25-04:00,True,4316724004,https://boards-api.greenhouse.io/v1/boards/sta...,"[{'id': 4025374004, 'name': 'Remote', 'locatio..."
3,Application Software Engineer,Applied Intution,https://boards.greenhouse.io/appliedintuition/...,"<div class=""content-intro""><p><strong>About Ap...",2023-03-21T14:50:45-04:00,False,4045084005,https://boards-api.greenhouse.io/v1/boards/app...,"[{'id': 4009577005, 'name': 'Stockholm', 'loca..."
4,Application Software Engineer,Applied Intution,https://boards.greenhouse.io/appliedintuition/...,"<div class=""content-intro""><p><strong>About Ap...",2023-03-21T14:50:45-04:00,False,4045011005,https://boards-api.greenhouse.io/v1/boards/app...,"[{'id': 4009261005, 'name': 'Seoul', 'location..."
...,...,...,...,...,...,...,...,...,...
227,Software Support Specialist (Weekends),Storable,https://boards.greenhouse.io/storable/jobs/481...,<p>Storable&nbsp;is seeking a&nbsp;Software Su...,2023-03-01T12:06:57-05:00,False,4813312004,https://boards-api.greenhouse.io/v1/boards/sto...,"[{'id': 4012074004, 'name': 'USA', 'location':..."
228,Software Engineer,Watershed,https://boards.greenhouse.io/watershed/jobs/42...,"<p><span style=""text-decoration: underline;"">R...",2023-02-14T18:53:40-05:00,False,4254100003,https://boards-api.greenhouse.io/v1/boards/wat...,"[{'id': 4018757003, 'name': 'Main Office', 'lo..."
229,"Software Engineer, Video Streaming",Flock Safety,https://boards.greenhouse.io/flocksafety/jobs/...,"<h2><span style=""color: #30d15e;""><strong>Comp...",2023-02-21T12:58:56-05:00,None,6627948002,https://boards-api.greenhouse.io/v1/boards/flo...,NaN
230,Software Engineer,Viam,https://boards.greenhouse.io/viamrobotics/jobs...,"<div class=""content-intro""><p>Viam is a roboti...",2023-02-10T13:14:31-05:00,False,4552131004,https://boards-api.greenhouse.io/v1/boards/via...,"[{'id': 4027040004, 'name': 'Bend, OR', 'locat..."


In [52]:
indeed_df

,job_title,company,link,description,date_posted,min_salary,max_salary,salary_type,city,country
0,Junior Web Developer,Easton Consulting Technologies,https://www.indeed.com/rc/clk?jk=22ae9f888a269...,"\nOverview\n Easton Consulting Technologies, L...","Fri, 24 Mar 2023 17:52:26 GMT",NaN,NaN,None,Remote,US
1,"Software and Systems Engineer, Junior",Booz Allen Hamilton,https://www.indeed.com/rc/clk?jk=e6ddeab1207e5...,\nThe Challenge:\n Are you looking for an oppo...,"Fri, 24 Mar 2023 14:32:13 GMT",52100.0,119000.0,YEAR,Chantilly,US
2,Junior Software Engineer,Riverside Research,https://www.indeed.com/rc/clk?jk=b84bf2b5de85b...,\n\n\n Riverside Research is an independent ...,"Fri, 24 Mar 2023 14:38:40 GMT",NaN,NaN,None,Dayton,US
3,Junior Level PHP Developer,American Partner Solutions,https://www.indeed.com/rc/clk?jk=74a08e2c81710...,\n\nIf you are a Jr. PHP Developer with experi...,"Fri, 24 Mar 2023 16:04:13 GMT",NaN,NaN,None,Tampa,US
4,Software Developer II,"SimVentions, Inc",https://www.indeed.com/rc/clk?jk=ee7d9e8fda4cc...,"\n\nSimVentions, consistently voted one Virgin...","Fri, 24 Mar 2023 18:13:30 GMT",NaN,NaN,None,Dahlgren,US
5,Junior Software Developer (On-Site),Datex Property Solutions,https://www.indeed.com/company/Datex-Property-...,We are looking for a Junior Software Developer...,"Fri, 24 Mar 2023 18:03:53 GMT",48750.0,-1.0,YEAR,Calabasas,US
6,Jr-Mid Level Data Engineer/Analyst with Front-...,Ryan Consulting Group,https://www.indeed.com/company/RYAN-Consulting...,"- Gather and create business requirements, und...","Fri, 24 Mar 2023 14:30:14 GMT",70000.0,80000.0,YEAR,Remote,US
7,Jr. Software Engineer - Direct Hire,Braintrust,https://www.indeed.com/rc/clk?jk=718afbac7ee12...,\nABOUT US:\n Braintrust is a user-owned talen...,"Fri, 24 Mar 2023 15:45:23 GMT",110000.0,125000.0,YEAR,San Francisco,US
8,Jr. .Net Developer,"OpenKnect, Inc.","https://www.indeed.com/company/OpenKnect,-Inc....","Atlanta, GA (Hybrid- LOCAL): Jr. .Net Develope...","Fri, 24 Mar 2023 15:39:46 GMT",20000.0,50000.0,YEAR,Atlanta,US
9,Jr .Net Developer,Oak Technologies Inc,https://www.indeed.com/company/Oak-Technologie...,JOB TITLE: .Net Developer LOCATION: hybrid (Re...,"Fri, 24 Mar 2023 13:35:23 GMT",90378.0,108842.0,YEAR,Dallas,US


In [53]:
dice_df

,title,company,link,description,date_posted,country,state,city,zip,remote
0,Junior Software Engineer,Lincoln Financial Group,https://www.dice.com/jobs/detail/Junior-Softwa...,Alternate Locations: Work from Home Work Arr...,2023-02-02T23:26:03.000Z,Radnor,PA,Radnor,19087,False
1,Jr Software Developer,Honu Services,https://www.dice.com/jobs/detail/Jr-Software-D...,Location: Nationwide with preference for DC me...,2023-03-19T12:13:16.000Z,None,None,None,None,True
2,Junior Software Developer,Nigel Frank International,https://www.dice.com/jobs/detail/Junior-Softwa...,Junior Software DeveloperI am currently workin...,2023-03-24T12:13:00.000Z,Bethesda,MD,Bethesda,20810,False
3,Junior Software Developer,Nigel Frank International,https://www.dice.com/jobs/detail/Junior-Softwa...,Junior Software DeveloperI am currently workin...,2023-03-17T12:10:45.000Z,Arlington,VA,Arlington,22201,False
4,Jr. Software Engineer- Embedded,"ConsultNet, LLC",https://www.dice.com/jobs/detail/Jr.-Software-...,Job DescriptionJr. Software Engineer- Embedded...,2023-03-03T22:22:19.000Z,Hawthorne,CA,Hawthorne,90250,False
5,Jr. Software Engineer (TS/SCI),MAXAR Technologies,https://www.dice.com/jobs/detail/Jr.-Software-...,Please review the job details below.Maxar is s...,2023-02-17T21:03:00.000Z,Melbourne,FL,Melbourne,32901,False
6,Junior Software Developer,Evoke Technologies,https://www.dice.com/jobs/detail/Junior-Softwa...,Position: Junior Software Developer Location: ...,2022-11-22T20:32:25.000Z,Dallas,TX,Dallas,75201,False
7,JUNIOR SOFTWARE ENGINEER,Incedo Inc,https://www.dice.com/jobs/detail/JUNIOR-SOFTWA...,Associate Software Engineer - Responsibilitie...,2023-03-01T17:28:38.000Z,Irving,TX,Irving,75014,False
8,Junior Software Engineer,Jobot,https://www.dice.com/jobs/detail/Junior-Softwa...,My client is a well known software company foc...,2023-03-16T00:23:03.000Z,MIDDLE CITY EAST,PA,MIDDLE CITY EAST,20001,False
9,Junior Software Engineer,Jobot,https://www.dice.com/jobs/detail/Junior-Softwa...,My client is a well known software company foc...,2023-03-16T00:23:03.000Z,Blacklick,OH,Blacklick,43004,False


In [54]:
greenhouse_df

,title,company,link,description,date,remote,greenhouse_id,greenhouse_api_url,location
0,"Software Engineer, Infrastructure",Loyal Health,https://boards.greenhouse.io/loyal/jobs/477271...,"<div class=""content-intro""><p>Loyal is an orga...",2023-03-08T18:14:34-05:00,False,4772718004,https://boards-api.greenhouse.io/v1/boards/loy...,"[{'id': 4009356004, 'name': 'Anywhere in the U..."
1,"Software Engineer, Backend",Janus Health,https://boards.greenhouse.io/janushealth/jobs/...,"<div class=""content-intro""><p>At Janus, we bel...",2023-03-11T01:37:04-05:00,True,4012385007,https://boards-api.greenhouse.io/v1/boards/jan...,"[{'id': 4000619007, 'name': 'Remote', 'locatio..."
2,Software Engineer,Stariwell,https://boards.greenhouse.io/stairwell/jobs/43...,"<div class=""content-intro""><h3>We’re expanding...",2022-06-02T14:21:25-04:00,True,4316724004,https://boards-api.greenhouse.io/v1/boards/sta...,"[{'id': 4025374004, 'name': 'Remote', 'locatio..."
3,Application Software Engineer,Applied Intution,https://boards.greenhouse.io/appliedintuition/...,"<div class=""content-intro""><p><strong>About Ap...",2023-03-21T14:50:45-04:00,False,4045084005,https://boards-api.greenhouse.io/v1/boards/app...,"[{'id': 4009577005, 'name': 'Stockholm', 'loca..."
4,Application Software Engineer,Applied Intution,https://boards.greenhouse.io/appliedintuition/...,"<div class=""content-intro""><p><strong>About Ap...",2023-03-21T14:50:45-04:00,False,4045011005,https://boards-api.greenhouse.io/v1/boards/app...,"[{'id': 4009261005, 'name': 'Seoul', 'location..."
...,...,...,...,...,...,...,...,...,...
227,Software Support Specialist (Weekends),Storable,https://boards.greenhouse.io/storable/jobs/481...,<p>Storable&nbsp;is seeking a&nbsp;Software Su...,2023-03-01T12:06:57-05:00,False,4813312004,https://boards-api.greenhouse.io/v1/boards/sto...,"[{'id': 4012074004, 'name': 'USA', 'location':..."
228,Software Engineer,Watershed,https://boards.greenhouse.io/watershed/jobs/42...,"<p><span style=""text-decoration: underline;"">R...",2023-02-14T18:53:40-05:00,False,4254100003,https://boards-api.greenhouse.io/v1/boards/wat...,"[{'id': 4018757003, 'name': 'Main Office', 'lo..."
229,"Software Engineer, Video Streaming",Flock Safety,https://boards.greenhouse.io/flocksafety/jobs/...,"<h2><span style=""color: #30d15e;""><strong>Comp...",2023-02-21T12:58:56-05:00,None,6627948002,https://boards-api.greenhouse.io/v1/boards/flo...,NaN
230,Software Engineer,Viam,https://boards.greenhouse.io/viamrobotics/jobs...,"<div class=""content-intro""><p>Viam is a roboti...",2023-02-10T13:14:31-05:00,False,4552131004,https://boards-api.greenhouse.io/v1/boards/via...,"[{'id': 4027040004, 'name': 'Bend, OR', 'locat..."


In [55]:
if not os.path.exists(f'./output/{today}'):
    os.mkdir(f'./output/{today}')

In [56]:
greenhouse_json = f'./output/{today}/greenhouse_{today}.json'
dice_json = f'./output/{today}/dice_{today}.json'
indeed_json = f'./output/{today}/indeed_{today}.json'

In [57]:
greenhouse_df.to_json(greenhouse_json, orient='records')
dice_df.to_json(dice_json, orient='records')
indeed_df.to_json(indeed_json, orient='records')

In [58]:
MONGO_URL = dotenv_values('config/config.env').get('MONGO_URL') 

In [59]:
client = pymongo.MongoClient(MONGO_URL)
db = client["test"]


In [60]:

Collection = db["greenhouse_test"]
 
# Opening the json file

with open(greenhouse_json) as file:
    file_data = json.load(file)
     
# Inserting the loaded data in the Collection
# if JSON contains data more than one entry
# insert_many is used else insert_one is used

if isinstance(file_data, list):
    Collection.insert_many(file_data) 
else:
    Collection.insert_one(file_data)



In [61]:
Collection = db["indeed_test"]
 
# Opening the json file

with open(indeed_json) as file:
    file_data = json.load(file)
     
# Inserting the loaded data in the Collection
# if JSON contains data more than one entry
# insert_many is used else insert_one is used

if isinstance(file_data, list):
    Collection.insert_many(file_data) 
else:
    Collection.insert_one(file_data)


In [62]:
Collection = db["dice_test"]
 
# Opening the json file

with open(dice_json) as file:
    file_data = json.load(file)
     
# Inserting the loaded data in the Collection
# if JSON contains data more than one entry
# insert_many is used else insert_one is used

if isinstance(file_data, list):
    Collection.insert_many(file_data) 
else:
    Collection.insert_one(file_data)
